***ANURAG***

Collaborative based filtering
1. Item based approach (go through various techniques like KNNs, etc.)


In [160]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
ratings=pd.read_csv('/content/drive/My Drive/datasets/Ratings .csv')
movies1=pd.read_csv('/content/drive/My Drive/datasets/movies.csv')
userdata=pd.read_csv('/content/drive/My Drive/datasets/user_data.csv')

In [ ]:
print('shape of ratings dataset ::',ratings.shape)
print('shape of movies dataset ::',movies.shape)
print('shape of userdata dataset ::',userdata.shape)

shape of ratings dataset :: (1000209, 4)
shape of movies dataset :: (3883, 3)
shape of userdata dataset :: (6040, 5)


In [ ]:
#Shape of datasets:
print('shape of ratings dataset ::',ratings.shape)
print('shape of movies dataset ::',movies.shape)
print('shape of userdata dataset ::',userdata.shape)

shape of ratings dataset :: (1000209, 4)
shape of movies dataset :: (3883, 3)
shape of userdata dataset :: (6040, 5)


In [ ]:
#merge datasets:: ratings,movies and userdata
d=pd.merge(userdata,ratings,on='UserID',how='left')
df=pd.merge(d,movies,on='MovieID',how='left')

In [ ]:
split_data,split_data.Genres = df.iloc[[i   for i,j in enumerate(df['Genres']) for value in j.split("|")] , :],[value for i in df['Genres'] for value in i.split("|")]
split_data.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genres
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,F,1,10,48067,661,3,978302109,James and the Giant Peach (1996),Animation
1,1,F,1,10,48067,661,3,978302109,James and the Giant Peach (1996),Children's
1,1,F,1,10,48067,661,3,978302109,James and the Giant Peach (1996),Musical
2,1,F,1,10,48067,914,3,978301968,My Fair Lady (1964),Musical
2,1,F,1,10,48067,914,3,978301968,My Fair Lady (1964),Romance
3,1,F,1,10,48067,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,F,1,10,48067,2355,5,978824291,"Bug's Life, A (1998)",Animation
4,1,F,1,10,48067,2355,5,978824291,"Bug's Life, A (1998)",Children's
4,1,F,1,10,48067,2355,5,978824291,"Bug's Life, A (1998)",Comedy


***Item Based Approach***

KNN

In [ ]:
!pip install pyreclab

***Experiment with Surprise SVD***

In [ ]:
ratings.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')

In [ ]:
aa=df.loc[df['UserID']==1]
aa.loc[aa['Rating'] == 4]
aa.head(15)

In [ ]:
!pip install surprise

     |████████████████████████████████| 6.5MB 3.1MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675718 sha256=99c7fb6b7dc6c902f753e28313c96d2eee6b72341a04fe307b383979e529fefe
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Reader, Dataset, SVD, model_selection
d = pd.read_csv('/content/drive/My Drive/datasets/Ratings .csv')
print(d.head)
reader = Reader()
data = Dataset.load_from_df(d[['UserID', 'MovieID', 'Rating']], reader)
#data.split(n_folds=5)

   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291 



In [ ]:
svd = SVD()
print("CROSS VALIDATING THE DATASET", '\n')
model_selection.cross_validate(svd, data, measures=['rmse'], verbose=True)
print("DONE CROSS VALIDATING THE DATASET",'\n')

print("TRAINING THE MODEL")
trainset = data.build_full_trainset()
svd.fit(trainset)
print("DONE TRAINING THE MODEL", '\n')

CROSS VALIDATING THE DATASET 

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8718  0.8719  0.8770  0.8718  0.8764  0.8738  0.0024  
Fit time          49.02   50.95   49.75   49.33   49.52   49.71   0.66    
Test time         2.39    3.09    2.38    2.41    2.41    2.54    0.28    
DONE CROSS VALIDATING THE DATASET 

TRAINING THE MODEL
DONE TRAINING THE MODEL 



In [ ]:
# r_ui -> The true rating
# iid  -> The item id (movie id)
# uid  -> The user id
print("Predicting rating for ",svd.predict(uid=3, iid=1000, r_ui=1))

Predicting rating for  user: 3          item: 1000       r_ui = 1.00   est = 3.28   {'was_impossible': False}


In [ ]:
from collections import defaultdict
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [ ]:
testset = trainset.build_anti_testset()

In [ ]:
predictions = svd.test(testset[20:50])

In [ ]:
predictions[1:2]

[Prediction(uid=1, iid=3107, r_ui=3.581564453029317, est=3.600571046956559, details={'was_impossible': False})]

In [ ]:
top_n = get_top_n(predictions, n=5)
top_n

defaultdict(list,
            {1: [(3147, 4.506135820674233),
              (589, 4.431506266150476),
              (110, 4.4083409265162965),
              (3035, 4.382649650214366),
              (1945, 4.3542264226177485)]})

In [ ]:
top_n = get_top_n(predictions, n=5)
# Print the recommended items for each user
print("PRINTING RECOMMENDATION")
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    print(uid)

PRINTING RECOMMENDATION
1 [3147, 589, 110, 3035, 1945]
1


***K-NN Using csr matrix***
*   ***Item based***

In [162]:
pivot_table=df.pivot_table(index='Title',columns='UserID',values='Rating').fillna(0)
pivot_table.head()

UserID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Night Mother (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [193]:
pivot_table.shape

(3706, 6040)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [178]:
distances, indices = model_knn.kneighbors(pivot_table.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 5)

In [191]:
qi = np.random.choice(movie_dfm.shape[0])
print('Similar Movies like :',qi, 'MoviID')
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(pivot_table.index[qi]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, pivot_table.index[indices.flatten()[i]], distances.flatten()[i]))

Similar Movies like : 1227 MoviID
Recommendations for Four Days in September (1997):

1: Executive Decision (1996), with distance of 0.6497563402372205:
2: Broken Arrow (1996), with distance of 0.6548010868570322:
3: Die Hard: With a Vengeance (1995), with distance of 0.656439938505968:
4: Breakdown (1997), with distance of 0.6655722904562306:


***Recommendation based On Cosine Similarity without algo***




In [ ]:
item_sim_df = pd.DataFrame(cosine_similarity(pivot2, pivot2), index=pivot2.index, columns=pivot2.index)
item_sim_df.head()

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",187 (1997),2 Days in the Valley (1996),20 Dates (1998),"20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010 (1984),24 7: Twenty Four Seven (1997),24-hour Woman (1998),28 Days (2000),3 Ninjas: High Noon On Mega Mountain (1998),3 Strikes (2000),"301, 302 (1995)","39 Steps, The (1935)","400 Blows, The (Les Quatre cents coups) (1959)",42 Up (1998),52 Pick-Up (1986),54 (1998),"7th Voyage of Sinbad, The (1958)",8 1/2 (1963),8 1/2 Women (1999),8 Heads in a Duffel Bag (1997),8 Seconds (1994),8MM (1999),A Chef in Love (1996),Abbott and Costello Meet Frankenstein (1948),"Abominable Snowman, The (1957)",About Adam (2000),About Last Night... (1986),...,Wonderland (1997),Wonderland (1999),Woo (1998),"Wood, The (1999)","Wooden Man's Bride, The (Wu Kui) (1994)",Working Girl (1988),"World Is Not Enough, The (1999)","World of Apu, The (Apur Sansar) (1959)","Wrong Man, The (1956)","Wrong Trousers, The (1993)",Wrongfully Accused (1998),Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X: The Unknown (1956),Xiu Xiu: The Sent-Down Girl (Tian yu) (1998),Yankee Zulu (1994),"Yards, The (1999)","Year My Voice Broke, The (1987)",Year of Living Dangerously (1982),Year of the Horse (1997),Yellow Submarine (1968),Yojimbo (1961),You Can't Take It With You (1938),You So Crazy (1994),You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kj�rlighetens kj�tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",1.000000,0.065338,0.030805,0.065478,0.048708,-0.001319,0.036612,0.176528,0.159973,0.075665,0.036112,0.021743,0.005995,0.006623,0.125345,0.079190,0.035304,0.018170,-0.002077,-0.002693,0.044022,0.128099,-0.001769,-0.002767,0.049727,0.000015,-0.008790,0.083803,0.053647,0.058235,0.003814,-0.005870,0.084214,-0.007693,0.020201,-0.004526,0.073389,-0.002731,-0.003982,0.070717,...,0.024357,-0.007451,0.053044,0.006107,-0.000951,0.066548,0.023457,-0.002404,-0.003547,0.035679,0.024929,0.026916,0.022303,0.033818,-0.003032,-0.007652,-0.001346,-0.007972,-0.004856,0.037398,-0.001805,0.075492,0.000349,0.077485,-0.003032,0.075338,0.110743,0.101890,0.043393,0.047054,0.030758,0.060574,-0.002833,0.035056,-0.001237,0.040590,0.024165,-0.001332,0.116574,0.009243
'Night Mother (1986),0.065338,1.000000,0.107374,0.096778,0.144480,-0.001834,0.046122,0.123378,0.074706,0.083988,0.013045,0.051613,0.089638,0.024013,0.043900,0.125092,0.058428,0.051779,0.052533,0.056134,0.034051,0.004726,-0.002460,0.041598,0.068722,0.084066,0.088665,0.106426,0.088644,0.046974,0.049550,-0.004708,0.056793,0.078636,0.082852,0.048011,0.046722,-0.003796,-0.005536,0.133571,...,0.102790,0.019844,-0.003963,0.010676,-0.001323,0.154700,-0.004511,0.022436,0.076762,0.027956,0.006211,0.065427,0.022342,0.008228,0.068762,0.074717,-0.001871,0.022086,0.189498,0.136072,-0.002509,0.091016,0.029323,0.090363,0.071935,0.052834,0.210143,0.094543,0.050218,0.014659,0.042061,0.065333,0.060202,0.124593,-0.001719,0.085001,0.054343,-0.001852,-0.005825,0.054699
'Til There Was You (1997),0.030805,0.107374,1.000000,0.082706,0.051965,0.079011,0.105672,0.091422,0.108952,0.054821,0.040220,0.012265,0.050856,0.080566,0.008710,0.186323,0.014024,0.012930,-0.002449,0.168083,0.103058,0.015852,-0.002087,0.029231,0.020222,-0.008749,0.001656,0.037813,0.095997,0.034908,0.006436,0.002951,0.064200,0.072818,0.103263,0.016085,0.015746,-0.003221,0.047597,0.1

In [ ]:
pivot2= pivot_table.apply(lambda x: x - np.nanmean(x), axis=1)
pivot2.head()

UserID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,...,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543,-0.018543
'Night Mother (1986),-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,...,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,-0.039073,2.960927,-0.039073,-0.039073,-0.039073,-0.039073
'Til There Was You (1997),-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,1.976821,-0.023179,-0.023179,...,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179,-0.023179
"'burbs, The (1989)",-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,3.853974,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,3.853974,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,1.853974,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,...,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,1.853974,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026,-0.146026
...And Justice for All (1979),-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,-0.122351,1.877649,-0.122351,-0.122351,-0.122351

In [ ]:
def get_similar_movie(Movie_name):
    if Movie_name not in pivot2.index:
        return None, None
    else:
        sim_movies = item_sim_df.sort_values(by=Movie_name, ascending=False).index[1:]
        sim_score = item_sim_df.sort_values(by=Movie_name, ascending=False).loc[:, Movie_name].tolist()[1:]
        return sim_movies, sim_score

In [ ]:
movies, score = get_similar_movie("'Til There Was You (1997)")
for x,y in zip(movies[:10], score[:10]):
  print(x,'-',y)

If Lucy Fell (1996) - 0.26732069622998056
Picture Perfect (1997) - 0.2557154439632222
To Gillian on Her 37th Birthday (1996) - 0.24105461392120084
Mad Love (1995) - 0.23132587558582302
Practical Magic (1998) - 0.22960163873809433
Something to Talk About (1995) - 0.22618112973033752
Circle of Friends (1995) - 0.22073534764102024
Beautician and the Beast, The (1997) - 0.22071668366822336
Evening Star, The (1996) - 0.2206349626920143
How to Make an American Quilt (1995) - 0.21623572955474876
